In [4]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from label_to_cat import LABEL_TO_CAT
from probs_saver import ProbStore

datapath = '../input/probs_resnet50_valid.h5'
save_to = '../input/resnet_val_comp.h5'

In [5]:
%%time
store = pd.HDFStore(datapath)
keys = store.keys()
print('{} tables in storage'.format(len(keys)))

1932 tables in storage
CPU times: user 7.37 s, sys: 100 ms, total: 7.47 s
Wall time: 7.47 s


In [6]:
cols = []
for i in range(1, 5271):
    cols.append(str(i))
    
saver = ProbStore(path=save_to)

for key in tqdm(keys): 
    table = store.select(key)
    assert table.shape[0] % 10 == 0
    two_cols = []
    sums_df = pd.DataFrame(columns=cols, dtype=np.float16)
    for i in range(int(table.shape[0] / 10)):
        st = 10 * i
        product_id = table.pr_id[st]
        img_num = table.img_num[st]
        sum1 = table.iloc[pd.np.r_[st : st + 4, st + 5:st + 9]].drop(['pr_id', 'img_num'], axis=1, inplace=False).sum()
        sum2 = table.iloc[pd.np.r_[st + 4: st + 5, st + 9:st + 10]].drop(['pr_id', 'img_num'], axis=1, inplace=False).sum()
        sum_ = (sum1 + sum2*3) / 14.
        #print(sum_)
        
        two_cols.append((product_id, img_num))
        sums_df = sums_df.append(sum_, ignore_index=True)
        #print(sums_df)
    two_cols_df = pd.DataFrame(data=two_cols, columns=['pr_id', 'img_num'], dtype=np.int32)
    df = pd.concat([two_cols_df, sums_df], axis=1)
    #print(df)
    saver.saveProbs(df)
    # debug
   # print(df)
    #break


100%|██████████| 1932/1932 [1:14:43<00:00,  2.32s/it]


# debug

In [7]:
%%time
comp_store = pd.HDFStore(save_to)
comp_keys = comp_store.keys()
print('{} tables in compressed val'.format(len(comp_keys)))

1932 tables in compressed val
CPU times: user 7 s, sys: 164 ms, total: 7.16 s
Wall time: 7.86 s


In [9]:
comp_store.select('d0').info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 256 entries, 0 to 255
Columns: 5272 entries, pr_id to 5270
dtypes: float16(5270), int32(2)
memory usage: 2.6 MB


In [12]:
t = comp_store.select('d0')
print(t.shape)
t.head(10)

(256, 5272)


,pr_id,img_num,1,2,3,4,5,6,7,8,...,5261,5262,5263,5264,5265,5266,5267,5268,5269,5270
0,13656822,0,0.0,5.960464e-08,0.0,1.192093e-07,5.960464e-08,7.152557e-07,0.0,3.576279e-07,...,0.0,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,5.364418e-07,1.251698e-06,5.960464e-08,6.294250e-05
1,13269598,0,0.0,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,...,0.0,0.000000e+00,0.0,5.364418e-07,0.000000e+00,0.000000e+00,0.000000e+00,1.668930e-06,2.980232e-07,4.768372e-07
2,3518877,0,0.0,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,...,0.0,0.000000e+00,0.0,5.364418e-07,0.000000e+00,0.000000e+00,0.000000e+00,1.668930e-06,2.980232e-07,4.768372e-07
3,8653221,0,0.0,0.000000e+00,0.0,0.000000e+00,5.960464e-07,0.000000e+00,0.0,0.000000e+00,...,0.0,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,1.788139e-07,0.000000e+00,1.192093e-07,0.000000e+00
4,3415644,0,0.0,5.364418e-07,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,...,0.0,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,1.192093e-07,3.337860e-06,0.000000e+00,4.768372e-07
5,13656822,3,0.0,6.616116e-06,0.0,1.132488e-06,5.364418e-07,1.788139e-07,0.0,1.788139e-07,...,0.0,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,5.435944e-05,9.536743e-07,5.960464e-07,5.066395e-06
6,4258965,0,0.0,1.013279e-05,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,...,0.0,1.788139e-07,0.0,0.000000e+00,0.000000e+00,5.960464e-08,0.000000e+00,8.940697e-07,3.576279e-07,2.062321e-05
7,4258965,1,0.0,6.735325e-06,0.0,9.536743e-07,0.000000e+00,0.000000e+00,0.0,0.000000e+00,...,0.0,5.960464e-08,0.0,0.000000e+00,3.576279e-07,1.072884e-06,2.384186e-07,4.768372e-07,3.576279e-07,8.106232e-06
8,3611994,1,0.0,5.960464e-08,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,...,0.0,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,5.960464e-07
9,13656822,1,0.0,3.576279e-07,0.0,3.039837e-06,5.960464e-07,0.000000e+00,0.0,7.748604e-07,...,0.0,0.000000e+00,0.0,0.000000e+00,1.192093e-07,0.000000e+00,2.503395e-06,2.384186e-07,2.205372e-06,1.847744e-06


In [14]:
x2 = np.array(t.drop(['pr_id', 'img_num'], axis=1, inplace=False).idxmax(axis=1), dtype=np.int)
x2.shape

(256,)

In [16]:
x2

array([2948, 1088, 1088, 1088, 1088, 3585, 5093, 5093, 4117, 1656, 1088,
        199, 1088, 1088, 1088, 1082,  457, 4487, 1370,  457, 1407,  908,
        457,  457, 1753,  457,  457,  457, 5064, 1527,  457,  457, 1527,
        457,  457,  457,  457,  457,  457,  516,  457,  457, 1407,  457,
        457,  457,  457,  457,  457,  457,  457, 1763, 1908,  807,  457,
       2598,  457,  457, 2549,  457,  457,  457,  457,  457,  457,  457,
        457,  457,  457,  457,  457,  457,  457,  457,  457,  457, 4487,
        457,  457,  457,  457,  457,  457,  457,  457,  457,  457,  457,
        457,  457,  456,  457,  457,  457,  457,  457,  457,  457,  457,
        607,  457,  457,  457,  556,  457,  457,  457,  457,  457,  755,
        457,  457,  457,  457,  457,  457,  457,  457,  457, 1170,  457,
        457,  457,  457,  457,  457,  457,  457,  457,  457,  778,  457,
        457,  457,  457,  457,  457,  457,  457,  457,  457,  457,  457,
        516,  457,  457,  457,  457,  457,  457,  4

In [17]:
t1 = store.select('d0')
print(t1.shape)
t1.head()

(2560, 5272)


,pr_id,img_num,1,2,3,4,5,6,7,8,...,5261,5262,5263,5264,5265,5266,5267,5268,5269,5270
0,13656822,0,0.0,1.192093e-07,0.0,5.960464e-08,0.000000e+00,4.172325e-07,0.0,5.960464e-08,...,0.0,0.0,0.0,0.000000e+00,0.0,0.0,1.788139e-07,1.370907e-06,5.960464e-08,0.000034
1,13656822,0,0.0,5.960464e-08,0.0,1.192093e-07,5.960464e-08,2.384186e-07,0.0,5.960464e-08,...,0.0,0.0,0.0,0.000000e+00,0.0,0.0,1.192093e-07,5.960464e-07,0.000000e+00,0.000025
2,13656822,0,0.0,5.960464e-08,0.0,4.172325e-07,1.192093e-07,1.966953e-06,0.0,4.768372e-07,...,0.0,0.0,0.0,1.192093e-07,0.0,0.0,1.966953e-06,1.668930e-06,1.192093e-07,0.000116
3,13656822,0,0.0,5.960464e-08,0.0,5.960464e-08,5.960464e-08,1.072884e-06,0.0,4.172325e-07,...,0.0,0.0,0.0,0.000000e+00,0.0,0.0,2.980232e-07,1.192093e-06,5.960464e-08,0.000175
4,13656822,0,0.0,5.960464e-08,0.0,5.960464e-08,5.960464e-08,8.940697e-07,0.0,2.384186e-07,...,0.0,0.0,0.0,0.000000e+00,0.0,0.0,1.788139e-07,1.728535e-06,5.960464e-08,0.000079


In [18]:
ids = []
for i in range(int(t1.shape[0] / 10)):
        product_id = t1.pr_id.iloc[10*i]
        id_ = int(t1[10*i: 10*i + 10].drop(['pr_id', 'img_num'], axis=1, inplace=False).sum().idxmax())
        ids.append(id_)
x1 = np.array(ids)
print(x1.shape)
x1

(256,)


array([2948, 1088, 1088, 1088, 1088, 3585, 5093, 5094, 4117, 1656, 1088,
        199, 1088, 1088, 1088, 1082,  457, 4487, 1230,  457, 1407,  908,
        457,  457, 2754,  457,  457,  457, 5064, 1527,  457,  457, 1527,
        457,  457,  457,  457,  457,  457,  516,  457,  457, 1407,  457,
        457,  457,  457,  457,  457,  457,  457, 2528, 1908,  807,  457,
       2598,  457,  457, 2549,  457,  457,  457,  457,  457,  457,  457,
        457,  457,  457,  457,  457,  457,  457,  457,  457,  457, 4487,
        457,  457,  457,  457,  457,  457,  457,  457,  457, 1527,  457,
        457,  457,  456,  457,  457,  457,  457,  457,  457,  457,  457,
        607,  457,  457,  457,  480,  457,  457,  457,  457,  457,  755,
        457,  457,  457,  457,  457,  457,  457,  457,  457, 1170,  457,
        457,  457,  457,  457,  457,  457,  457,  457,  457,  778,  457,
        457,  457,  457,  457,  457,  457,  457,  457,  457,  457,  457,
        516,  457,  457,  457,  457,  457,  457,  4

In [19]:
x1 == x2

array([ True,  True,  True,  True,  True,  True,  True, False,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
       False,  True,  True,  True,  True,  True, False,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True, False,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True, False,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True, False,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,